In [3]:
# %pip install pyproj
#%pip install geopandas
import pyproj
import geopandas as gpd
import pandas as pd
from pyproj import Transformer

# 변환기 설정: EPSG:5179 -> EPSG:4326
transformer = Transformer.from_crs("EPSG:5186", "EPSG:4326", always_xy=True)

# 파일 경로 및 데이터 로드
file_path = '출동현황.csv'
data = pd.read_csv(file_path, encoding='utf-8')

# TM 좌표를 위도/경도로 변환
data['경도'], data['위도'] = transformer.transform(data['GIS_X좌표'], data['GIS_Y좌표'])

# 변환 결과 저장
output_data = data[['GIS_X좌표', 'GIS_Y좌표', '경도', '위도']]

# 결과 출력
print(output_data.head())
output_data.to_csv('출동현황_위도경도_변환.csv', index=False, encoding='utf-8-sig')

       GIS_X좌표      GIS_Y좌표          경도         위도
0  203672.7963  544819.5166  127.041537  37.502834
1  204086.2300  545674.7700  127.046218  37.510539
2  205250.7300  545167.9700  127.059385  37.505966
3  205702.2559  544627.7565  127.064488  37.501096
4  202013.5147  545584.4746  127.022774  37.509732


In [35]:
# from pyproj import Transformer

# # 좌표계 후보
# epsg_codes = ["EPSG:5174", "EPSG:5175", "EPSG:5176", "EPSG:5177", "EPSG:5178", "EPSG:5179"]

# # 테스트 좌표
# x, y = 203672.7963, 544819.5166

# # 결과 저장
# for epsg_code in epsg_codes:
#     transformer = Transformer.from_crs(epsg_code, "EPSG:4326", always_xy=True)
#     lng, lat = transformer.transform(x, y)
#     print(f"{epsg_code}: 경도={lng}, 위도={lat}")

EPSG:5174: 경도=127.04282808547671, 위도=38.4065133497207
EPSG:5175: 경도=127.04257467015763, 위도=37.95606660302001
EPSG:5176: 경도=129.04264601117254, 위도=38.40652811282575
EPSG:5177: 경도=131.04246619583552, 위도=38.40654608370887
EPSG:5178: 경도=119.64365263998462, 위도=24.666943133726956
EPSG:5179: 경도=119.64593848163594, 위도=24.66416588685903


In [61]:
import json


In [17]:
import pandas as pd
import folium
import json

csv_file_path = '인구밀도2.csv'
geojson_file_path = 'Seoul_Gu.json'
population_data = pd.read_csv(csv_file_path, encoding='utf-8')
population_data['인구 (명)'] = population_data['인구 (명)'].str.replace(',', '').astype(float)

district_population = population_data.groupby('동별(2)')['인구 (명)'].sum().reset_index()
district_population.rename(columns={'동별(2)': 'SIG_KOR_NM', '인구 (명)': 'population_density'}, inplace=True)

with open(geojson_file_path, 'r', encoding='EUC-KR') as file:
    geojson_data = json.load(file)

center = [37.541, 126.986]  # 서울 중심 좌표
m = folium.Map(location=center, zoom_start=11)

#구별 인구
folium.Choropleth(
    geo_data=geojson_data,
    data=district_population,
    columns=['SIG_KOR_NM', 'population_density'],
    key_on='properties.SIG_KOR_NM',
    fill_color='PuBu',
    legend_name='구별 인구 (명)'
).add_to(m)

m.save('구별_인구수_map.html')
m

In [15]:
import pandas as pd
import folium
import json

csv_file_path = '인구밀도2.csv'
geojson_file_path = 'Seoul_Gu.json'
population_data = pd.read_csv(csv_file_path, encoding='utf-8')
population_data['인구 (명)'] = population_data['인구 (명)'].str.replace(',', '').astype(float)

district_population = population_data.groupby('동별(2)')['인구 (명)'].sum().reset_index()
district_population.rename(columns={'동별(2)': 'SIG_KOR_NM', '인구 (명)': 'population_density'}, inplace=True)

with open(geojson_file_path, 'r', encoding='EUC-KR') as file:
    geojson_data = json.load(file)

center = [37.541, 126.986]  # 서울 중심 좌표
m = folium.Map(location=center, zoom_start=11)

#센터 마커
data = pd.read_csv('소방서_위치정보.csv', encoding='cp949')
station = data.loc[:, ["서ㆍ센터명", "위도", "경도"]]
for index, row in station.iterrows():
    folium.Marker(
    [row["위도"], row["경도"]], popup = row["서ㆍ센터명"],
    icon = folium.Icon(icon="fa-solid fa-fire-extinguisher", prefix ="fa-solid", color = "red")
    ).add_to(m)

#구별 인구
folium.Choropleth(
    geo_data=geojson_data,
    data=district_population,
    columns=['SIG_KOR_NM', 'population_density'],
    key_on='properties.SIG_KOR_NM',
    fill_color='PuBu',
    legend_name='구별 인구 (명)'
).add_to(m)

m.save('구별_인구수_센터_map.html')
m

In [12]:
import pandas as pd
import folium
import json

csv_file_path = '인구밀도2.csv'
geojson_file_path = 'Seoul_Gu.json'
population_data = pd.read_csv(csv_file_path, encoding='utf-8')
population_data['인구 (명)'] = population_data['인구 (명)'].str.replace(',', '').astype(float)

district_population = population_data.groupby('동별(2)')['인구 (명)'].sum().reset_index()
district_population.rename(columns={'동별(2)': 'sggnm', '인구 (명)': 'population_density'}, inplace=True)

with open(geojson_file_path, 'r', encoding='EUC-KR') as file:
    geojson_data = json.load(file)

data = pd.read_csv('인구밀도2.csv', encoding='utf-8')
station = data.loc[:, ["위도", "경도","개수"]]   
#icon_list = ['fa-solid fa-6','fa-solid fa-5','fa-solid fa-4''fa-solid fa-3','fa-solid fa-2']
center = [37.541, 126.986]  # 서울 중심 좌표
m = folium.Map(location=center, zoom_start=11)

def get_marker_icon(count):
    if count == 6:
        return "6"
    elif count == 5:
        return "5"
    elif count == 4:
        return "4"
    elif count == 3:
        return "3"
    elif count == 2:
        return "2"
    else:
        return "info-sign"

#센터 마커
for index, row in station.iterrows():
    folium.Marker(
        [row["위도"], row["경도"]],
        icon=folium.Icon(icon=get_marker_icon(row["개수"]), prefix="fa",color = "red")  
    ).add_to(m)

#구별 인구
folium.Choropleth(
    geo_data=geojson_data,
    data=district_population,
    columns=['sggnm', 'population_density'],
    key_on='properties.SIG_KOR_NM',
    fill_color='PuBu',
    legend_name='구별 인구 (명)'
).add_to(m)

m.save('구별_인구수_센터_구별개수_map.html')

m
